In [ ]:
#!wget -O dataset.zip https://www.dropbox.com/s/3zfz89hik6989h5/dataset.zip?dl=1
!wget -O dataset_four.zip https://www.dropbox.com/s/0v6rdf3xhoge0vh/unet_color_quatre.zip?dl=1 
#!wget -O dataset_gt.zip https://www.dropbox.com/s/wvjlpqykck6jwc6/unet_color_tres.zip?dl=1
#!wget -O dataset_gros.zip https://www.dropbox.com/s/e7ndtiqce74gqli/sub.zip?dl=1
 
#!unzip dataset.zip
!unzip dataset_four.zip > /dev/null
#!unzip dataset_gt.zip
#!unzip dataset_gros.zip

--2020-12-04 09:17:19--  https://www.dropbox.com/s/0v6rdf3xhoge0vh/unet_color_quatre.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.1, 2620:100:601c:1::a27d:601
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/0v6rdf3xhoge0vh/unet_color_quatre.zip [following]
--2020-12-04 09:17:19--  https://www.dropbox.com/s/dl/0v6rdf3xhoge0vh/unet_color_quatre.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc3591d9964906b1b3816349c126.dl.dropboxusercontent.com/cd/0/get/BEYxssxcWt3JbdczRv7-bPfTg-UUe3aH3bJW5RUiNl-Vc1MAnTY4s5z8-gxY8fGs83Ww8ONTQVSwPIjyd8gtO7wY6PC1R-fpuK5NR7EqZM9YpPymLSLsVViS1JXbNta6fLA/file?dl=1# [following]
--2020-12-04 09:17:20--  https://uc3591d9964906b1b3816349c126.dl.dropboxusercontent.com/cd/0/get/BEYxssxcWt3JbdczRv7-bPfTg-UUe3aH3bJW5RUiNl-Vc1MAnTY4s5z8-gxY8fGs83Ww8ONTQVSwPIjyd8gtO

In [ ]:
# %tensorflow_version 1.x
import io
from datetime import datetime

from matplotlib import pyplot as plt

from numpy.random import seed
import tensorflow as tf
from tensorflow.keras import metrics
import keras
import keras.backend as K
from PIL import Image


import numpy as np
import cv2


from model import *
from data import *

seed(1)
#print(tf.__version__)

In [ ]:
test_path_root = "./unet_color_quatre/test/image/"
test_path = test_path_root + "*.png"
train_path = './unet_color_quatre/train'
generator_path = None #'./generator/'
tb_dir = "./log/" + datetime.now().strftime("%Y/%m/%d-%H:%M:%S")
# tb_cb = keras.callbacks.TensorBoard(tb_dir, write_graph=True, update_freq=10)


data_gen_args = dict(rotation_range=0.2,
						width_shift_range=0.05,
						height_shift_range=0.05,
						shear_range=0.05,
						zoom_range=0.05,
						horizontal_flip=True,
						fill_mode='nearest')

myGene = trainGenerator(1, train_path,'image','label',data_gen_args,save_to_dir = generator_path, mask_color_mode = "rgb", decode_flag=True, target_size=(512,512)) #AQui

model = get_small_unet(n_filters=64, bn = True, input_size=(512,512,1), output_channels = 3, loss_func = dice_coef_loss) ### AqUI!!!!!
# model_checkpoint = ModelCheckpoint('unet_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
#tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names=True)



Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 512, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 512, 512, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 512, 512, 64) 36928       batch_normalization[0][0]        
_______________________________________________________________________________________

In [ ]:
#model.fit_generator(myGene,steps_per_epoch=300,epochs=10) # 300, 5
model.fit(myGene,steps_per_epoch=300,epochs=10) # 300, 5

testGene = testGenerator(test_path, target_size=(512,512))
#results = model.predict_generator(testGene,16)
results = model.predict(testGene,16)

Found 33 images belonging to 1 classes.
Found 33 images belonging to 1 classes.
Epoch 1/10
300/300 [==============================] - 93s 311ms/step - loss: 0.2396 - categorical_accuracy: 0.4431
Epoch 2/10
300/300 [==============================] - 94s 313ms/step - loss: 0.2246 - categorical_accuracy: 0.4129
Epoch 3/10
300/300 [==============================] - 94s 312ms/step - loss: 0.2211 - categorical_accuracy: 0.4073
Epoch 4/10
300/300 [==============================] - 94s 313ms/step - loss: 0.2188 - categorical_accuracy: 0.4029
Epoch 5/10
300/300 [==============================] - 93s 310ms/step - loss: 0.2173 - categorical_accuracy: 0.3975
Epoch 6/10
300/300 [==============================] - 93s 310ms/step - loss: 0.2168 - categorical_accuracy: 0.4036
Epoch 7/10
300/300 [==============================] - 93s 309ms/step - loss: 0.2154 - categorical_accuracy: 0.4162
Epoch 8/10
300/300 [==============================] - 93s 310ms/step - loss: 0.2149 - categorical_accuracy: 0.4192


In [ ]:
#model.summary()

### IOU
---


In [ ]:
testGene = testGenerator(test_path, target_size=(512,512))

results = model.predict(testGene,16) #tocat



In [ ]:
def get_iou(ground, prediction, th):
  assert ground.shape[2] == prediction.shape[2]
  
  ground = ground.astype(np.float32) / ground.max()

  ious = [0] * ground.shape[2]

  for channel_idx in range(0, ground.shape[2]):
    channel_gt = ground[:,:,channel_idx]
    channel_pred = np.copy(prediction[:,:,channel_idx])
    
    channel_pred = cv2.resize(channel_pred, (channel_gt.shape[1], channel_gt.shape[0]), interpolation = cv2.INTER_AREA) 

    channel_pred[channel_pred <= th] = 0
    channel_pred[channel_pred > th] = 255

    aux = np.zeros_like(channel_gt)

    intersection = cv2.bitwise_and(channel_gt, channel_pred)
    intersection_area = np.count_nonzero(intersection)

    union = cv2.bitwise_or(channel_gt, channel_pred)
    union_area = np.count_nonzero(union)

    ious[channel_idx] += round(intersection_area / union_area, 3)
    
  return ious

In [ ]:
def iou(y_true, y_pred):
    # y_pred = K.cast(K.greater(y_pred, .5), dtype='float32') # .5 is the threshold
    
    inter = K.sum(K.abs(y_true * y_pred), axis=-1)
    union = ((K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1)) - inter)
    return 1 - (inter / union)

alpha = 0.5

ious = []
for idx, res in enumerate(results):
  gt_image = cv2.imread("./unet_color_quatre/test/label/" + str(idx) + ".png")

  gt_image = decode(gt_image)

  iou = get_iou(gt_image, res, alpha)
  ious.append(iou)
  if idx < 10:
    print("Image 0"+ str(idx) + ": " + str(iou))
  else:
    print("Image "+ str(idx) + ": " + str(iou))

print("###########################################")
print("Mean: ", np.mean(ious, axis = 0))

Image 00: [0.013, 0.78, 0.036]
Image 01: [0.0, 0.617, 0.043]
Image 02: [0.003, 0.774, 0.011]
Image 03: [0.0, 0.63, 0.045]
Image 04: [0.054, 0.697, 0.13]
Image 05: [0.054, 0.475, 0.121]
Image 06: [0.002, 0.761, 0.056]
Image 07: [0.048, 0.694, 0.029]
Image 08: [0.0, 0.756, 0.068]
Image 09: [0.04, 0.7, 0.044]
Image 10: [0.002, 0.604, 0.09]
Image 11: [0.0, 0.723, 0.026]
Image 12: [0.008, 0.699, 0.018]
Image 13: [0.001, 0.753, 0.0]
Image 14: [0.0, 0.753, 0.021]
Image 15: [0.053, 0.684, 0.11]
Image 16: [0.03, 0.727, 0.0]
###########################################
Mean:  [0.01811765 0.69570588 0.04988235]


In [ ]:
res = results[2]
plt.figure(1,(16,12))
for res in results: 
  for i in range(0,3):
  #plt.subplot(1,4, i+1)
    r = res[:,:,i]
    r[r < 0.5] = 0
    r[r >= 0.5] = 1
  #plt.imshow(res[:,:,i]);

<Figure size 1152x864 with 0 Axes>

In [ ]:
import os

!rm -rf out/
!mkdir out

for idx, r in enumerate(results):
  for c_idx in range(0, r.shape[2]):
    path = "./out/" + str(c_idx) + "/"
    if not os.path.exists(path): 
      os.makedirs(path)

    cv2.imwrite(path + str(idx) + ".png", r[:,:,c_idx]*255)

In [ ]:
from google.colab import files

!zip -r ./out.zip ./out > /dev/null
files.download("./out.zip");

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>